### Load libraries & config

In [ ]:
from datetime import date, datetime
import girder_client as gc
import json
import os
from urllib.parse import quote
import random
import re
import string
import tzlocal
from urllib.request import urlopen

with open("config.json", "r") as fp:
    config=json.load(fp)

### Connect to Girder

In [ ]:
girder_connection = gc.GirderClient(
    apiUrl="{}/api/v1/".format(
        config["girder-production"]["host"]
    )
)
girder_connection.authenticate(
    username=config["girder-production"]["user"],
    password=config["girder-production"]["password"]
)

In [ ]:
activity_sets = girder_connection.get(
    "folder?parentType=collection&parentId={}".format(
        girder_connection.get(
            "collection?name=Volumes"
        )[0]["_id"]
    )
)

### Authorize Viewers to View their Users as Authorized in Admin Panel

In [ ]:
for a in activity_sets:
    print("Activity Set: {}".format(a['name']))
    for v in a['meta']['members']['viewers']:
        print("Viewer: {}".format(girder_connection.getUser(v)['firstName']))
        for u in a['meta']['members']['viewers'][v]:
            try: 
                access = girder_connection.get("user/{}/access".format(u))
            except:
                access = {'users': []}
            print("User: {}".format(girder_connection.getUser(u)['firstName']))
            print("access: {}".format(", ".join([user['name'] for user in access['users']])))
            if (v not in [user['id'] for user in access['users']]):
                access['users'].append({'id': v, 'level': 0})
                girder_connection.put("user/{}/access".format(u),
                    data={"access":json.dumps(access)}
                )
            try:
                responses = girder_connection.get(
                    "folder?parentType=user&parentId={}&name=Responses".format(
                        u
                    )
                )
                for responses_folder in responses:
                    print("Responses: {}".format(responses_folder['name'] if 'name' in responses_folder else responses_folder))
                    try:
                        access = girder_connection.get(
                            "folder/{}/access".format(
                                responses_folder['_id']
                            )
                        )
                        if v not in [uid["id"] for uid in access['users']]:
                            access['users'].append({'id': v, 'level': 0})
                        girder_connection.put(
                            'folder/{}/access'.format(
                                responses_folder['_id']
                            ),
                            data={
                                "access":json.dumps(access),
                                "recurse":False
                            },
                        )
                        print(access)
                        activity_set_responses = girder_connection.get(
                            "folder?parentType=folder&parentId={}&name={}".format(
                                responses_folder['_id'],
                                a['name']
                            )
                        )
                        for activity_set_response_folder in activity_set_responses: 
                            print("Responses Folder: {}".format(activity_set_response_folder['name']))
                            access = girder_connection.get(
                                "folder/{}/access".format(
                                    activity_set_response_folder['_id']
                                )
                            )
                            print("access: {}".format(", ".join([uid["name"] for uid in access['users']])))
                            if v not in [uid["id"] for uid in access['users']]:
                                access['users'].append({'id': v, 'level': 0})
                            girder_connection.get('folder/{}'.format(activity_set_response_folder['_id']))['name']
                            girder_connection.put(
                                'folder/{}/access'.format(
                                    activity_set_response_folder['_id']
                                ),
                                data={
                                    "access":json.dumps(access),
                                    "recurse":True
                                },
                            )
                    except:
                        activity_set_responses = None
            except:
                responses = None
        print("\n")

### Authorize Managers to View Users as Authorized in Admin Panel

In [ ]:
for a in activity_sets:
    for u in a['meta']['members']['users']:
        try:
            access = girder_connection.get(
                'user/{}/access'.format(u)
            )
            for m in a['meta']['members']['managers']:
                if m not in [uid['id'] for uid in access['users']]:
                    print(m)
                    access['users'].append({'id': m, 'level': 0})
            print(access)
            girder_connection.put(
                'user/{}/access'.format(u),
                data={"access":json.dumps(access)}
            )
        except:
            access = {}

### Authorize Healthy Brain Network Data Access Group Data Access to All **EMA: Parent** Activity Data

In [ ]:
for user in [a for a in activity_sets if a['name'] == "EMA: Parent"][0]['meta']['members']['users']:
    try:
        responses = girder_connection.get(
            "folder?parentType=user&parentId={}&name=Responses".format(
                user
            )
        )
        for responses_folder in responses:
            try:
                ema_parent_responses = girder_connection.get(
                    "folder?parentType=folder&parentId={}&name=EMA%3A%20Parent".format(
                        responses_folder['_id']
                    )
                )
                for ema_parent_response_folder in ema_parent_responses: 
                    access = girder_connection.get(
                        "folder/{}/access".format(
                            ema_parent_response_folder['_id']
                        )
                    )
                    if "5bab8fc80a76ac049becd5ec" not in [gid["id"] for gid in access['groups']]:
                        access['groups'].append({'id': '5bab8fc80a76ac049becd5ec', 'level': 0})
                print(access)
                girder_connection.put(
                    'folder/{}/access'.format(
                        ema_parent_response_folder['_id']
                    ),
                    data={
                        "access":json.dumps(access),
                        "recurse":True
                    },
                )
            except:
                ema_parent_responses = None
    except:
        responses = None

### Consolidate duplicated permissions

In [ ]:
for u in girder_connection.get('user'):
    access = girder_connection.get(
        'user/{}/access'.format(u['_id'])
    )
    new_access = {'groups': access['groups'], 'users': []}
    for au in access['users']:
        if au['id'] not in [uid['id'] for uid in new_access['users']]:
            new_access['users'].append(au)
    print(new_access)
    girder_connection.put(
        'user/{}/access'.format(u["_id"]),
        data={"access":json.dumps(new_access)}
    )

### Purge non-existent users from admin panel

In [ ]:
all_assignees = list({
    *{
        e for a in
        activity_sets for e in
        a['meta']['members']['editors']
    },
    *{
        m for a in
        activity_sets for m in
        a['meta']['members']['managers']
    },
    *{
        u for a in
        activity_sets for u in
        a['meta']['members']['users']
    },
    *{
        v for a in
        activity_sets for v in
        a['meta']['members']['viewers']
    },
    *{
        u for a in
        activity_sets for v in
        a['meta']['members']['viewers'] for u in
        a['meta']['members']['viewers'][v] 
    }
})
purge_list = []
for assignee in all_assignees:
    try:
        girder_connection.getUser(assignee)
    except:
        purge_list.append(assignee)
print(purge_list)
for a in activity_sets:
    original_members = a['meta']['members'].copy()
    for role in ['editors', 'users', 'managers']:
        a['meta']['members'][role] = list(set(a['meta']['members'][role]) - set(purge_list))
    for purgee in purge_list:
        a['meta']['members']['viewers'].pop(purgee, None)
    for v in a['meta']['members']['viewers']:
        a['meta']['members']['viewers'][v] = list(set(a['meta']['members']['viewers'][v]) - set(purge_list))
    if(original_members != a['meta']['members']):
        girder_connection.put(
            'folder/{}'.format(
                a['_id']
            ),
            data={
                'metadata':json.dumps(
                    a['meta']
                )
            }
        )

In [ ]:
for manager in [a for a in activity_sets if a['name'] == "EMA: Parent"][0]['meta']['members']['managers']:
    u = girder_connection.getUser(manager)
    print("{} {}".format(u['firstName'], u['lastName']))

In [ ]:
all_assignees = list({
    *{
        e for a in
        activity_sets for e in
        a['meta']['members']['editors']
    },
    *{
        m for a in
        activity_sets for m in
        a['meta']['members']['managers']
    },
    *{
        u for a in
        activity_sets for u in
        a['meta']['members']['users']
    },
    *{
        v for a in
        activity_sets for v in
        a['meta']['members']['viewers']
    },
    *{
        u for a in
        activity_sets for v in
        a['meta']['members']['viewers'] for u in
        a['meta']['members']['viewers'][v] 
    }
})
for assignee in all_assignees:
    if assignee not in [a for a in activity_sets if a['name'] == "EMA: Parent"][0]['meta']['members']['users']:
        u = girder_connection.getUser(assignee)
        print("{} {} {}".format(u['firstName'], u['lastName'], u['_id']))

In [ ]:
print(len([
            "5b7f1db1e534fe32c5f5db8c",
            "5c2e3d3dc25b5a6a94d60067",
            "5c2e30e6c25b5a6a94d60066",
            "5bcf3462336da80de9145728",
            "5c3515ada0e4760df6a033ec",
            "5bbdd20f01a25f0dd6703f1a",
            "5c2e47c2c25b5a6a94d6006c",
            "5c3516c7a0e4760df6a033ed",
            "5c3633afa0e4760df6a033fc",
            "5bd7b7af336da80de9145a53",
            "5c2e4697c25b5a6a94d6006b",
            "5bdb6a97336da80de9145e35",
            "5b634cbf0fc13a0df633ae00",
            "5c2e2fe7c25b5a6a94d60065",
            "5bcf785b336da80de9145744",
            "5bd08ceb336da80de91457ea",
            "5c2d286ec25b5a6a94d5fffd",
            "5b634cc20fc13a0df633ae05",
            "5c37a61c4a3ec00def0c81b3",
            "5bd08d18336da80de91457eb",
            "5be4994a336da80de91468f7",
            "5bd87d09336da80de9145b23",
            "5b4e1f1f8f5b186be929676d",
            "5c36692da0e4760df6a03426",
            "5c1a69aac25b5a6a94d5fbd8",
            "5bd08898336da80de91457a5",
            "5c50b1664e9a670deb99b4e3",
            "5bd8696d336da80de9145a5a",
            "5b7f1d7ae534fe32c5f5db8b",
            "5bd731d9336da80de9145a02",
            "5be31566336da80de91464df",
            "5bd26081336da80de9145896",
            "5b7f1cf7e534fe32c5f5db8a",
            "5c37a5bf4a3ec00def0c81b2",
            "5bcf3e49336da80de9145729",
            "5bdb5260336da80de9145dfa",
            "5b4ca32338488311497dc9bc",
            "5bc4e3fe01a25f0dd6703f97"
        ]))
for i, u in enumerate([
            "5b7f1db1e534fe32c5f5db8c",
            "5c2e3d3dc25b5a6a94d60067",
            "5c2e30e6c25b5a6a94d60066",
            "5bcf3462336da80de9145728",
            "5c3515ada0e4760df6a033ec",
            "5bbdd20f01a25f0dd6703f1a",
            "5c2e47c2c25b5a6a94d6006c",
            "5c3516c7a0e4760df6a033ed",
            "5c3633afa0e4760df6a033fc",
            "5bd7b7af336da80de9145a53",
            "5c2e4697c25b5a6a94d6006b",
            "5bdb6a97336da80de9145e35",
            "5b634cbf0fc13a0df633ae00",
            "5c2e2fe7c25b5a6a94d60065",
            "5bcf785b336da80de9145744",
            "5bd08ceb336da80de91457ea",
            "5c2d286ec25b5a6a94d5fffd",
            "5b634cc20fc13a0df633ae05",
            "5c37a61c4a3ec00def0c81b3",
            "5bd08d18336da80de91457eb",
            "5be4994a336da80de91468f7",
            "5bd87d09336da80de9145b23",
            "5b4e1f1f8f5b186be929676d",
            "5c36692da0e4760df6a03426",
            "5c1a69aac25b5a6a94d5fbd8",
            "5bd08898336da80de91457a5",
            "5c50b1664e9a670deb99b4e3",
            "5bd8696d336da80de9145a5a",
            "5b7f1d7ae534fe32c5f5db8b",
            "5bd731d9336da80de9145a02",
            "5be31566336da80de91464df",
            "5bd26081336da80de9145896",
            "5b7f1cf7e534fe32c5f5db8a",
            "5c37a5bf4a3ec00def0c81b2",
            "5bcf3e49336da80de9145729",
            "5bdb5260336da80de9145dfa",
            "5b4ca32338488311497dc9bc",
            "5bc4e3fe01a25f0dd6703f97"
        ]):
    print(i)
    user = girder_connection.getUser(u)
    print("{} {} ({})".format(
        user["firstName"],
        user["lastName"],
        user["login"]
    ))

In [ ]:
len([
            "5b7f1db1e534fe32c5f5db8c",
            "5c2e3d3dc25b5a6a94d60067",
            "5c2e30e6c25b5a6a94d60066",
            "5bcf3462336da80de9145728",
            "5c3515ada0e4760df6a033ec",
            "5bbdd20f01a25f0dd6703f1a",
            "5c2e47c2c25b5a6a94d6006c",
            "5c3516c7a0e4760df6a033ed",
            "5c3633afa0e4760df6a033fc",
            "5bd7b7af336da80de9145a53",
            "5c2e4697c25b5a6a94d6006b",
            "5bdb6a97336da80de9145e35",
            "5b634cbf0fc13a0df633ae00",
            "5c2e2fe7c25b5a6a94d60065",
            "5bcf785b336da80de9145744",
            "5bd08ceb336da80de91457ea",
            "5c2d286ec25b5a6a94d5fffd",
            "5b634cc20fc13a0df633ae05",
            "5c37a61c4a3ec00def0c81b3",
            "5bd08d18336da80de91457eb",
            "5be4994a336da80de91468f7",
            "5bd87d09336da80de9145b23",
            "5b4e1f1f8f5b186be929676d",
            "5c36692da0e4760df6a03426",
            "5c1a69aac25b5a6a94d5fbd8",
            "5bd08898336da80de91457a5",
            "5c50b1664e9a670deb99b4e3",
            "5bd8696d336da80de9145a5a",
            "5b7f1d7ae534fe32c5f5db8b",
            "5bd731d9336da80de9145a02",
            "5be31566336da80de91464df",
            "5bd26081336da80de9145896",
            "5b7f1cf7e534fe32c5f5db8a",
            "5c37a5bf4a3ec00def0c81b2",
            "5bcf3e49336da80de9145729",
            "5bdb5260336da80de9145dfa",
            "5b4ca32338488311497dc9bc",
            "5bc4e3fe01a25f0dd6703f97"
])

In [ ]:
[x for x in """5b4ca32338488311497dc9bc	
5b4e1f1f8f5b186be929676d	
5bd08898336da80de91457a5	
5b634cc20fc13a0df633ae05	
5bd08d18336da80de91457eb	
5bd8696d336da80de9145a5a	
5bd7b7af336da80de9145a53	
5bd08ceb336da80de91457ea	
5bcf3462336da80de9145728	
5bd731d9336da80de9145a02	
5bcf785b336da80de9145744	
5bcf3e49336da80de9145729	
5bbdd20f01a25f0dd6703f1a	
5b7f1cf7e534fe32c5f5db8a	
5b7f1d7ae534fe32c5f5db8b	
5b9ebaece534fe07ad95e29a	
5b634cbe0fc13a0df633adff	
5b634cbf0fc13a0df633ae00	
5b7f1db1e534fe32c5f5db8c	
5bc4e3fe01a25f0dd6703f97	
5bd87d09336da80de9145b23	
5bd26081336da80de9145896	
5bdb6a97336da80de9145e35	
5bdb5260336da80de9145dfa	
5be31566336da80de91464df	
5be4994a336da80de91468f7	
5c1a69aac25b5a6a94d5fbd8	
5c2d286ec25b5a6a94d5fffd	
5c2e2fe7c25b5a6a94d60065	
5c2e30e6c25b5a6a94d60066	
5c2e3d3dc25b5a6a94d60067	
5c2e4697c25b5a6a94d6006b	
5c2e47c2c25b5a6a94d6006c	
5c3515ada0e4760df6a033ec	
5c3516c7a0e4760df6a033ed	
5c3633afa0e4760df6a033fc	
5c36692da0e4760df6a03426	
5c37a5bf4a3ec00def0c81b2	
5c37a61c4a3ec00def0c81b3	
5c50b1664e9a670deb99b4e3""".split("""	\n""") if x not in [
            "5b7f1db1e534fe32c5f5db8c",
            "5c2e3d3dc25b5a6a94d60067",
            "5c2e30e6c25b5a6a94d60066",
            "5bcf3462336da80de9145728",
            "5c3515ada0e4760df6a033ec",
            "5bbdd20f01a25f0dd6703f1a",
            "5c2e47c2c25b5a6a94d6006c",
            "5c3516c7a0e4760df6a033ed",
            "5c3633afa0e4760df6a033fc",
            "5bd7b7af336da80de9145a53",
            "5c2e4697c25b5a6a94d6006b",
            "5bdb6a97336da80de9145e35",
            "5b634cbf0fc13a0df633ae00",
            "5c2e2fe7c25b5a6a94d60065",
            "5bcf785b336da80de9145744",
            "5bd08ceb336da80de91457ea",
            "5c2d286ec25b5a6a94d5fffd",
            "5b634cc20fc13a0df633ae05",
            "5c37a61c4a3ec00def0c81b3",
            "5bd08d18336da80de91457eb",
            "5be4994a336da80de91468f7",
            "5bd87d09336da80de9145b23",
            "5b4e1f1f8f5b186be929676d",
            "5c36692da0e4760df6a03426",
            "5c1a69aac25b5a6a94d5fbd8",
            "5bd08898336da80de91457a5",
            "5c50b1664e9a670deb99b4e3",
            "5bd8696d336da80de9145a5a",
            "5b7f1d7ae534fe32c5f5db8b",
            "5bd731d9336da80de9145a02",
            "5be31566336da80de91464df",
            "5bd26081336da80de9145896",
            "5b7f1cf7e534fe32c5f5db8a",
            "5c37a5bf4a3ec00def0c81b2",
            "5bcf3e49336da80de9145729",
            "5bdb5260336da80de9145dfa",
            "5b4ca32338488311497dc9bc",
            "5bc4e3fe01a25f0dd6703f97"
]]